# First, set up the training images

## Import data
Replace `<path_to_data>` with the path to where you placed the NPY training files.

In [ ]:
path = <path_to_data>

In [ ]:
import numpy as np
import pandas as pd
X = np.load(path+'X.npy')
y = np.load(path+'y.npy')
file = np.load(path+'file.npy')

# set numpy seed for model
np.random.seed(0) # for reproducibility

In [ ]:
# convert to grayscale
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

X = rgb2gray(X)
# Reshape to match CNN model requirements
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

# Define CNN model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
# Convolutional Neural Network https://machinelearningmastery.com/keras-functional-api-deep-learning/

# Convolutional Neural Network
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, AveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Sequential


input_shape = (X.shape[1], X.shape[2], X.shape[3])

nClasses = 2

def createModel():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(nClasses, activation='sigmoid'))
    
    return model



In [ ]:
######### train test split ###############
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from keras.utils import to_categorical
from sklearn.utils import class_weight
# set up k-fold cross-val
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)
for in1, in2 in skf.split(X, y):
  print(len(in1))

5346
5346
5346
5347
5347


# Define MLP model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

# define and fit scaler
scaler = preprocessing.StandardScaler()


################## define and run model itself, after you've found good params ########
mlp = MLPClassifier(hidden_layer_sizes= (100, 50),
                    activation = 'relu',
                    solver = 'lbfgs',
                    alpha = 1e-5,
                    learning_rate = 'constant',
                    random_state = 0, max_iter=5000)

pipe = Pipeline(steps =[('scaler',scaler) , ('MLPClassifier', mlp)])

# Split train/test with k-fold

In [ ]:
######### train test split ###############
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from keras.utils import to_categorical
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE


# set up k-fold cross-val
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)

######################
# Set training process params, class weights (get class weights from the train/test split, below)
batch_size = 64
epochs = 80

number_models= 5

###################
# construct metrics
metrics = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
###################


# full list of models and histories
models=list()
histories=list()
histories_all = list()

# outcomes divided by fold
y_test_byfold = list()
y_train_byfold = list()
X_test_byfold = list()
X_train_byfold = list()
file_test_byfold = list()
file_train_byfold = list()
file_val_byfold = list()

# lists for CNN models
histories_byfold = list()
models_byfold = list()

# list for MLP models
pipes_byfold = list()
#####################
# construct "fold" counter to save data
fold = 0

for train_index, test_index in skf.split(X, y):
    print(fold)

    # reset model and history counter
    models = list()
    histories = list()

    # split data
    X1, X_test = X[train_index], X[test_index]
    y1, y_test = y[train_index], y[test_index]
    file1, file_test = file[train_index], file[test_index]

    # construct validation set
    X_train, X_val, y_train, y_val, file_train, file_val = train_test_split(X1, y1, file1, stratify = y1, test_size=0.15, random_state=0)
    
    # append all data to lists, to save for later
    y_train_byfold.append(y_train)
    y_test_byfold.append(y_test)

    X_test_byfold.append(X_test)
    X_train_byfold.append(X_train)

    file_test_byfold.append(file_test)
    file_train_byfold.append(file_train)
    file_val_byfold.append(file_val)


##################### ##################### ##################### 
    # MLP model
##################### ##################### ##################### 

   # resample training data
    over = SMOTE(sampling_strategy = .1)
    under = RandomUnderSampler(sampling_strategy=1)
    X_train_mlp, y_train_mlp = over.fit_resample(X_train, y_train)
    X_train_mlp, y_train_mlp = under.fit_resample(X_train, y_train)

    for i in range(0,5):
        pipe.fit(X_train, y_train)
        pipes.append(pipe)
    pipes_byfold.append(pipes)

##################### ##################### ##################### 
    # CNN MODEL
##################### ##################### ##################### 

    # compute class weights an assign to variable for model below
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)
    weight = {0: class_weights[0], 1: class_weights[1]}
    print(weight)

    # scale Xs and convert Ys to categorical
    X_train, X_val, X_test = X_train/255, X_val/255, X_test/255
    y_train, y_val, y_test = to_categorical(y_train), to_categorical(y_val), to_categorical(y_test)



    # run ensemble CNN model for each kfold
    for i in range(0,number_models):
      print(i)
      model = createModel()
      model.compile(optimizer='adamax', loss=keras.losses.BinaryCrossentropy(), metrics=metrics)
      history = model.fit(X_train, y_train,
                          batch_size=batch_size, 
                          epochs=epochs, verbose=1, class_weight = weight,
                          validation_data = (X_val, y_val),
                          # callbacks=[early_stopping], # this stops the training when the metric listed in model.compile has stopped improving
                          shuffle=False)
      
      # append to revolving models and histories list, in order to later append to full list of models and history
      models.append(model)
      histories.append(history.history)
      # append all histories for each model in each fold, to plot later
      histories_all.append(history.history)

    histories_byfold.append(histories)
    models_byfold.append(models)

    # iterate "fold" counter plus one
    
    fold = fold + 1



0
{0: 0.5264133456904542, 1: 9.964912280701755}
0
Epoch 1/80
71/71 [==============================] - 5s 38ms/step - loss: 0.6778 - tp: 2121.1111 - fp: 321.4583 - tn: 2013.6528 - fn: 214.0000 - accuracy: 0.8762 - precision: 0.8528 - recall: 0.9199 - auc: 0.9234 - val_loss: 0.3282 - val_tp: 669.0000 - val_fp: 136.0000 - val_tn: 666.0000 - val_fn: 133.0000 - val_accuracy: 0.8323 - val_precision: 0.8311 - val_recall: 0.8342 - val_auc: 0.9318
Epoch 2/80
71/71 [==============================] - 2s 24ms/step - loss: 0.5115 - tp: 1539.6250 - fp: 794.0694 - tn: 1541.0417 - fn: 795.4861 - accuracy: 0.6667 - precision: 0.6665 - recall: 0.6671 - auc: 0.8005 - val_loss: 0.3205 - val_tp: 669.0000 - val_fp: 131.0000 - val_tn: 671.0000 - val_fn: 133.0000 - val_accuracy: 0.8354 - val_precision: 0.8363 - val_recall: 0.8342 - val_auc: 0.9313
Epoch 3/80
71/71 [==============================] - 2s 24ms/step - loss: 0.5039 - tp: 1566.8611 - fp: 766.1944 - tn: 1568.9167 - fn: 768.2500 - accuracy: 0.6835 - p

In [ ]:
# save histories, testing data, histories, and models

import pickle
for fold in range(0,5):
  for model in range(0,5):
    path = '/content/drive/MyDrive/COLAB/saved_models/ensemblemodel_' + str(fold) + '-' + str(model)
    models_byfold[fold][model].save(path)
  
  path = "/content/drive/MyDrive/COLAB/saved_models/X_train_" + str(fold) + ".dat"
  with open(path, "wb") as f:
    pickle.dump(X_train_byfold[fold], f)

  path = "/content/drive/MyDrive/COLAB/saved_models/X_test_" + str(fold) + ".dat"
  with open(path, "wb") as f:
    pickle.dump(X_test_byfold[fold], f)
  
  path = "/content/drive/MyDrive/COLAB/saved_models/Y_test_" + str(fold) + ".dat"
  with open(path, "wb") as f:
    pickle.dump(y_test_byfold[fold], f)
  
  path = "/content/drive/MyDrive/COLAB/saved_models/Y_train_" + str(fold) + ".dat"
  with open(path, "wb") as f:
    pickle.dump(y_train_byfold[fold], f)

with open("/content/drive/MyDrive/COLAB/saved_models/ensemblemodel_histories_byfold.dat", "wb") as f:
  pickle.dump(histories_byfold, f)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensemblemodel_0-0/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensemblemodel_0-1/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensemblemodel_0-2/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensemblemodel_0-3/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensemblemodel_0-4/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensemblemodel_1-0/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensemblemodel_1-1/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensemblemodel_1-2/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensemblemodel_1-3/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/COLAB/saved_models/ensem

In [ ]:
# save the models
import pickle as pkl

path  = '/Users/jtollefs/Documents/SOCIOLOGY/github/MGP_circle_detection/ML_training/saved_models/'
file = path+'mlp_models_byfold.pkl'
with open(file, 'wb') as f:
    pkl.dump(pipes_byfold, f)